In [1]:
import sys
sys.path += ['../..']
sys.path += ['..']

from data_collection.data_collection import LoggerSet, Logger

import numpy as np
import pandas as pd
import plotly.express as px
from data_collection.video_data import get_frame_iterator
from pathlib import Path
from typing import Iterable, Tuple, List
from tqdm import tqdm
import datetime
import tensorflow as tf
import tensorflow.keras as keras

%load_ext autoreload
%autoreload 2


2024-07-02 20:41:41.780626: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-02 20:41:42.398231: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Note

# Loading data 

In [4]:
from with_linear_acc_v2 import Session
sessions_v0 = Session.load_multiple_session('../data/v0',         
    camera_log_name = 'PicameraV2', 
    angular_speed_control_log_name='AngularSpeedControlV2'
)
sessions_v30Jun = Session.load_multiple_session('../data/v30Jun')


[mov,mp4,m4a,3gp,3g2,mj2 @ 0x58e585e55b40] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x58e5866f8640] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x58e5898eef80] moov atom not found


In [3]:
def data_prep(obj: Session):
    # main
    obj.add_col_frame_idx('angular_speed_control_df', 'camera_df', time_offset_ms=200)
    obj.add_col_rolling_mean_nominmax(
        'angular_speed_control_df', 
        'angular_velocity', 'angular_velocity_smooth',
        window=50, shift=-25, center=False 
        )

def sample_prep(obj: Session):
    frames, df = obj.data['frames'], obj.data['angular_speed_control_df']

    # include only the row during driving
    df = df.query('speed>0')

    # exclude the row with non-existing frame indices
    df = df.query(f'offset_frame_idx<{len(frames)}')

    # the output is the mean smoothed angular velocity during the frame around 200ms before 
    outputs = df.groupby('offset_frame_idx')[['angular_velocity_smooth']].mean().dropna()

    # match the frames to the outputs
    frames_reindexed = frames[outputs.index.values]    

    #return frames_reindexed, outputs

    obj.samples = (frames_reindexed, outputs/300)

def stack_samples(objs: List[Session], sample_axis=0):
    samples_stacked = []
    for s in zip(*[o.samples for o in objs]):
        samples_stacked.append(np.concatenate(s, axis=sample_axis))
    return samples_stacked


In [5]:
[data_prep(s) for s in sessions_v30Jun+sessions_v0]; 
[sample_prep(s) for s in sessions_v30Jun+sessions_v0]; 

x, y = stack_samples(sessions_v30Jun)

from train_tools import validation_chunk_split
train_idx, val_idx = validation_chunk_split(len(x), val_split=0.2)
train_x, train_y = x[train_idx], y[train_idx]
val_x, val_y = x[val_idx], y[val_idx]


In [7]:
x_val2, y_val2 = stack_samples(sessions_v0)


## Inspection only

In [8]:
df_vis = sessions[0].data['angular_speed_control_df']

In [52]:

def mean_nominmax(x):
    return (x.sum()-x.max()-x.min())/(len(x)-2)
asd0 = df_vis['angular_velocity'].rolling(100).apply(mean_nominmax)#.shift(-99)

asd = df_vis['angular_velocity'].rolling(50).apply(mean_nominmax).shift(-25)
asd1 = df_vis['angular_velocity'].rolling(3).apply(mean_nominmax).shift(-2)


px.line(y=[asd1, asd, asd0])

In [54]:
px.line(df_vis, y=['angular_velocity_smooth', 'angular_velocity'])

In [83]:
px.imshow(sessions[0].data['frames'][-1000])

# Model

In [9]:
def get_model(lr=.1, other_metrics=[]):
    tf.keras.backend.clear_session()
    image_shape = 64, 114, 3

    img_aug_preprocess_layers = [
        keras.layers.RandomTranslation(0.05, 0.05, fill_mode='reflect'),
        keras.layers.RandomRotation(0.02, fill_mode='reflect'),
        keras.layers.RandomZoom(0.05, fill_mode='reflect'),
        keras.layers.RandomContrast(0.3),
        keras.layers.RandomBrightness(factor=0.6, value_range=[0, 1])
    ]
    
    layers =  [
        keras.layers.Conv2D(16, 5, strides=2, activation='relu'), 
        keras.layers.Conv2D(32, 5, strides=2, activation='relu'), 
        keras.layers.Conv2D(64, 5, strides=2, activation='relu'),         
        keras.layers.Conv2D(64, (5, 3), activation='relu'), 
        keras.layers.Flatten(),
        keras.layers.Dropout(0.2),
        
        keras.layers.Dense(64, activation='relu'), 
        keras.layers.Dense(64, activation='relu'), 
        keras.layers.Dense(1), 
    ]
    
    model = keras.Sequential(
        [
            keras.layers.InputLayer(image_shape),
            keras.layers.Rescaling(1/255), 
            *img_aug_preprocess_layers,
            *layers, 
        ])


    optimiser = keras.optimizers.Adam(lr)
    model.compile(optimizer=optimiser, loss='Huber', metrics=['MAE']+other_metrics)

    return model 

model = get_model()
model.summary()

2024-07-02 20:43:34.785907: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-02 20:43:34.818458: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-02 20:43:34.818700: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 64, 114, 3)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_translation              │ (None, 64, 114, 3)     │             0 │
│ (RandomTranslation)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation                 │ (None, 64, 114, 3)     │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_zoom (RandomZoom)        │ (None, 64, 114, 3)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_contrast                 │ (None, 64, 114, 3)     │             0 │
│ (RandomContrast)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_brightness               │ (None, 64, 114, 3)     │             0 │
│ (RandomBrightness)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 30, 55, 16)     │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 26, 32)     │        12,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 5, 11, 64)      │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 1, 9, 64)       │        61,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 576)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 576)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 167,969 (656.13 KB)

 Trainable params: 167,969 (656.13 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
from train_tools import find_lr, tg_notify, use_tensorboard, end_epoch_notify

In [42]:
find_lr(get_model(), train_x, train_y)
#tensorboard --logdir ./logs --bind_all

Epoch 1/100
162/162 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - MAE: 0.1154 - loss: 0.0133 - learning_rate: 1.0000e-06
Epoch 2/100
162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - MAE: 0.1110 - loss: 0.0128 - learning_rate: 1.1220e-06
Epoch 3/100
162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - MAE: 0.1136 - loss: 0.0137 - learning_rate: 1.2589e-06
Epoch 4/100
162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - MAE: 0.1096 - loss: 0.0128 - learning_rate: 1.4125e-06
Epoch 5/100
162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - MAE: 0.1094 - loss: 0.0130 - learning_rate: 1.5849e-06
Epoch 6/100
162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - MAE: 0.1097 - loss: 0.0128 - learning_rate: 1.7783e-06
Epoch 7/100
162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - MAE: 0.1096 - loss: 0.0131 - learning_rate: 1.9953e-06
Epoch 8/100
162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - MAE: 0.1081 - loss: 0.0125 - learning_rate: 2.2387e-06
Epoch 9/100
162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - MAE: 0.1099 - loss: 0.0130 - learning_rate: 2.5119e-06
Epoch 10/1

In [11]:
model = get_model(3e-4)
model.fit(
    train_x,
    train_y,
    epochs=100, 
    validation_data = (val_x, val_y),
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),
        use_tensorboard('training'), 
        end_epoch_notify()
        ] 
    )

Epoch 1/100


2024-07-02 20:43:49.028416: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


146/146 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - MAE: 0.1149 - loss: 0.0136 - val_MAE: 0.1093 - val_loss: 0.0126
Epoch 2/100
146/146 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - MAE: 0.1134 - loss: 0.0133 - val_MAE: 0.1175 - val_loss: 0.0134
Epoch 3/100
146/146 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - MAE: 0.1125 - loss: 0.0129 - val_MAE: 0.1084 - val_loss: 0.0124
Epoch 4/100
146/146 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - MAE: 0.1094 - loss: 0.0123 - val_MAE: 0.1144 - val_loss: 0.0129
Epoch 5/100
146/146 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - MAE: 0.1119 - loss: 0.0129 - val_MAE: 0.0990 - val_loss: 0.0110
Epoch 6/100
146/146 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - MAE: 0.1090 - loss: 0.0124 - val_MAE: 0.1012 - val_loss: 0.0117
Epoch 7/100
146/146 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - MAE: 0.1069 - loss: 0.0119 - val_MAE: 0.0999 - val_loss: 0.0105
Epoch 8/100
146/146 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - MAE: 0.1045 - loss: 0.0115 - val_MAE: 0.1044 - val_loss: 0.0116
Epoch 9/100
146/146 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - MAE:

In [ ]:
#frames_reindexed#, other_inputs_reindexed, outputs

# visualise the performance

In [12]:

#train_frames, train_other, train_out
val_idxv = np.sort(val_idx)
train_idxv = np.sort(train_idx)

val_y_pred = model.predict(x[val_idxv], batch_size=64)
train_y_pred = model.predict(x[train_idxv], batch_size=64)


val_outv = y[val_idxv]
train_outv = y[train_idxv]

px.line(np.c_[train_outv[:, :], train_y_pred[:, :]], title='train')

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [13]:
px.line(np.c_[val_outv[:, :], val_y_pred[:, :]], title='train')

In [46]:
#model.export('29Jun-export.keras')
model.save('2Jul.keras')


In [14]:
y_val2_pred = model.predict(x_val2, batch_size=64)


215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [16]:
px.line(np.c_[y_val2[:, :], y_val2_pred[:, :]], title='val2')